## Práctica 3

### Propiedades estadísticas del lenguaje

## Objetives

1. Comprobar si las *stopwords* que encontramos en paqueterias de *NLP* coinciden con las palabras más comúnes obtenidas en Zipf
2. Comprobar si Zipf se cumple para un lenguaje artificial creado por ustedes

In [ ]:
%pip install numpy
%pip install nltk
%pip install wordcloud
%pip install matplotlib

## Parte 1

### Comprobación de Zipf para el español

1. Comprobar si las *stopwords* que encontramos en paqueterias de *NLP* coinciden con las palabras más comúnes obtenidas en Zipf
    - Utilizar el [corpus CREA](https://corpus.rae.es/frec/CREA_total.zip)
    - Realizar una nube de palabras usando las stopwords de paqueteria y las obtenidas através de Zipf
    - Responder las siguientes preguntas:
        - ¿Obtenemos el mismo resultado? Si o no y ¿Porqué?
    

In [ ]:
import pandas as pd

# (rank, word, frequency, normalized_frequency)
data = {"rank": [], "word": [], "frequency": [], "normalized_frequency": []}

FILENAME = "crea.txt"
file = open(FILENAME, "r", encoding="latin-1")

# skip header line
file.readline()

for line in file:
    striped_line = line.strip()
    raw_rank, word, raw_frequency, raw_normalized_frequency = striped_line.split()
    rank = int(float(raw_rank))
    frequency = int(raw_frequency.replace(",", ""))
    normalized_frequency = float(raw_normalized_frequency)
    data["rank"].append(rank)
    data["word"].append(word)
    data["frequency"].append(frequency)
    data["normalized_frequency"].append(normalized_frequency)

# close file
file.close()

# convert data to pandas DataFrame
dataset = pd.DataFrame(data)
dataset

In [ ]:
"""
verificar que las k = 10 primeras palabras
sean stop words
comparar con el español de nltk
"""
import nltk
from nltk.corpus import stopwords

# print("k = 10 primeras palabras")
# print([word for rank, word, *_ in dataset if rank < 10])

nltk.download('stopwords')
nltk.download('punkt')

print(f"stop words en español de nltk \n{stopwords.words('spanish')[:10]}\n")

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt


top_10_words = dataset.sort_values(by='frequency', ascending=False).head(10)
word_freq = dict(zip(top_10_words['word'], top_10_words['frequency']))
wordcloud = WordCloud(width=800, height=400).generate_from_frequencies(word_freq)

plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import cess_esp, stopwords
from nltk.probability import FreqDist
from wordcloud import WordCloud

nltk.download('cess_esp')

words = cess_esp.words()

spanish_stopwords = stopwords.words('spanish')

# Create a frequency distribution of the words in the corpus
freq_dist = FreqDist(words)

# Get the frequency of each stopword in the corpus
stopword_freq = {word: freq_dist[word] for word in spanish_stopwords[:10]}

# Create a word cloud
wordcloud = WordCloud(width=800, height=400).generate_from_frequencies(stopword_freq)

# Display the word cloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import nltk

from nltk.corpus import cess_esp, stopwords
from nltk.probability import FreqDist
from wordcloud import WordCloud

nltk.download('cess_esp')

words = cess_esp.words()

spanish_stopwords = stopwords.words('spanish')

# Create a frequency distribution of the words in the corpus
freq_dist = FreqDist(words)

# Get the frequency of each stopword in the corpus
stopword_freq = {word: freq_dist[word] for word in spanish_stopwords[:10]}

# Create a word cloud
wordcloud = WordCloud(width=800, height=400).generate_from_frequencies(stopword_freq)

# Display the word cloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

## Parte 2

### Zipf para un lenguaje artificial

2. Comprobar si Zipf se cumple para un lenguaje artificial creado por ustedes
    - Deberán darle un nombre a su lenguaje
    - Mostrar una oración de ejemplo
    - Pueden ser una secuencia de caracteres aleatorios
    - Tambien pueden definir el tamaño de las palabras de forma aleatoria



In [ ]:
import string
import numpy as np
import string

"""
alternativa uno
para crear un lenguaje
vamos a utilizar un enfoque probabilístico

a partir de los distintos 26 caracteres de ascii que son letras
aleatoriamente vamos a generar k cadenas de longitud 1, 2, ..., n
con k = 25 y n = 16
"""

# random + language = ranguage
LANGUAGUE_NAME = "Ranguage"
MAX_FREQUENCY = 25
MAX_WORD_LENGTH = 16

lexicon = set()

for i in range(MAX_WORD_LENGTH + 1):
    # we won't generate words of length 0 or 1
    if i == 0 or i == 1: continue

    lexicon_at_size_i = []
    for _ in range(MAX_FREQUENCY):
        word = np.random.choice(list(string.ascii_lowercase), i)
        lexicon_at_size_i.append(''.join(word))

    lexicon.update(lexicon_at_size_i)

lexicon = list(lexicon)

# print first 10 words
print("randomly generated words in the lexicon")
print(lexicon[:10])
print()

# randomly pick 6-10 words from the lexicon to generate a sentence
# generate 5 sentences

print("randomly generated sentences")
sentences = [
    " ".join(np.random.choice(lexicon, np.random.randint(6, 11))) for _ in range(5)
]

for sentence in sentences:
    print(sentence)

In [ ]:
"""
generar aleatoriamente un corpus de texto del lenguaje que hemos creado.
una concatenación de párrafos de longitud aleatoria

un corpus va a ser un conjunto de párrafos, y cada párrafo va a ser un conjunto de oraciones
"""

def generate_random_sentence():
    amount = np.random.randint(6, 11)
    return " ".join(np.random.choice(lexicon, amount))

def generate_random_paragraph():
    amount = np.random.randint(3, 7)
    return ", ".join([generate_random_sentence() for _ in range(amount)])

def generate_random_document():
    amount = np.random.randint(10, 25)
    return "\n\n".join([generate_random_paragraph() for _ in range(amount)])

def generate_corpus():
    amount = np.random.randint(3, 10)
    return [generate_random_document() for _ in range(amount)]

corpus = generate_corpus()
corpus

In [ ]:
from collections import defaultdict

"""
encontrar palabras únicas del corpus
obtener la frecuencia de cada palabra
"""

word_to_frequency = defaultdict(int)
for document in corpus:
    for paragraph in document.split("\n\n"):
        for sentence in paragraph.split(","):
            for word in sentence.split(" "):
                word_to_frequency[word] += 1

lexicon = [ key for key in word_to_frequency.items() ]
len(lexicon)

In [ ]:
"""
graficar palabra por frecuencia
"""

import matplotlib.pyplot as plt

plt.bar(range(len(lexicon)), [ value for key, value in word_to_frequency.items() ])
plt.show()
